In [ ]:
import requests
import re
import os

from logging import getLogger
from config import YandexEnvELZ
import time
import sys

In [ ]:
folder_id = YandexEnvELZ.folder
model = "yandexgpt"
api_key = YandexEnvELZ.api_key

In [ ]:
def calc_prompt_tokens(text, folder_id, api_key):
    prompt = {"modelUri": f"gpt://{folder_id}/{model}/latest", "text": text}
    url_tokenize = "https://llm.api.cloud.yandex.net/foundationModels/v1/tokenize"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Api-Key {api_key}",
        "x-data-logging-enabled": "false",
    }
    #
    response = requests.post(url_tokenize, headers=headers, json=prompt)
    text_new = re.sub(r"true", r"True", response.text)
    text_new = re.sub(r"false", r"False", text_new)

    return len(eval(text_new)["tokens"])

In [ ]:
def yandex_generate(prompt_list):
    
    temperature= 0
    max_tokens = 500    

        
    messages_prompt = [
            {"role": "system", "text": prompt_list[0]},
            {"role": "user", "text": prompt_list[1]},  
            #1 пример
            {"role": "user", "text": prompt_list[2]},  
            {"role": "assistant", "text": prompt_list[3]},
            #2 пример
            {"role": "user", "text": prompt_list[4]},  
            {"role": "assistant", "text": prompt_list[5]},
            #3 пример
            {"role": "user", "text": prompt_list[6]},  
            {"role": "assistant", "text": prompt_list[7]},
            #4 пример
            {"role": "user", "text": prompt_list[8]},  
            {"role": "assistant", "text": prompt_list[9]},
            #5 пример
            {"role": "user", "text": prompt_list[10]},  
            {"role": "assistant", "text": prompt_list[11]},

            {"role": "user", "text": prompt_list[12]} 
            
        ]

    prompt = {
        "modelUri": f"gpt://{folder_id}/{model}/latest",
        "completionOptions": {
            "stream": False,
            "temperature": temperature,
            "maxTokens": max_tokens,
        },
        "messages": messages_prompt,
    }
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Api-Key {api_key}",
        "x-data-logging-enabled": "false",
    }

    response = requests.post(url, headers=headers, json=prompt)  
    print(response.text)
    result = eval(response.text)["result"]["alternatives"][0]["message"]["text"]
    
    
    return result
yandex_generate(prompt_list)

### dataset

In [ ]:
import os
import re
import numpy as np 
import pandas as pd 
import pickle, json, string, torch
from datetime import datetime

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, set_seed
from datasets import Dataset, DatasetDict, load_dataset
from transformers import BitsAndBytesConfig,HfArgumentParser,TrainingArguments, logging, DataCollatorForLanguageModeling, Trainer
from transformers import DataCollatorForTokenClassification

In [ ]:
dataset = load_dataset("MalakhovIlya/NEREL")
folders = ["train", "test", "dev"]


#@title функции для изъятия сущностей
#!!! - то, что было в изначальном коде
def get_list_values(text):
    return text.split()

def replc_t_n(text):
    return re.sub("\t|\n", " ", text).strip()

def grouped_and_sort_labeled_data(annotation_file):

    # df_ann = pd.DataFrame([get_list_values(replc_t_n(i)) for i in annotation_file if ";" not in i])
    df_ann = pd.DataFrame([get_list_values(i) for i in annotation_file  if ";" not in i])
    df_ann[2] = df_ann[2].astype("int")
    df_ann[3] = df_ann[3].astype("int")


    '''
    df_ann.to_csv("df_ann.csv")
    # (1869-окончание) - если группировать, то исчезает прокуратура Бишкека как сущность
    # есть отдельно прокуратура и отдельно Бишкек
    #!!! grouped = df_ann.groupby([1, 2])[3].min().reset_index()
    grouped.to_csv("grouped.csv")
    print(grouped)
    '''
    return df_ann.sort_values(by=2)[[1,2,3]].values #!!! grouped.sort_values(by=2)[[1,2,3]].values

def split_text_on_labeled_tokens(text, labels):
    def chunk_text_labeling(text, start, end, is_ner = False):
        # print(f"start = {start}, end = {end}\n  text[start: end]={text[start: end]}")
        chunk_iter = 0
        ner_chunk = text[start: end].split()
        for part_of_chunk in ner_chunk:
            split_text.append(part_of_chunk)
            if is_ner:
                if chunk_iter == 0:
                    ner_label.append("B-"+ner)
                else:
                    ner_label.append("I-"+ner)
                chunk_iter += 1
            else:
                ner_label.append("O")
    ### inner function

    init_start = 0
    split_text = []
    ner_label = []
    for ner, start, end in labels:
        if start > init_start:

            chunk_text_labeling(text, init_start, start)
            chunk_text_labeling(text, start, end, True)
        else:
            chunk_text_labeling(text, start, end, True)
        init_start = end

    return split_text, ner_label


#@title функции для очистки токенов

''' в разделенных словах есть знаки препинания - нужно убрать
split_text_sent = ['"В', 'ходе', 'проведения', 'оперативно-профилактического', 'мероприятия', 'под', 'кодовым', 'названием', '"', 'Арсенал', '"', 'в',
'''

def starts_or_ends_with_punctuation(s):
    punct=string.punctuation
    punct+=' '
    return [(len(s)!=1 and s[0] in punct), (len(s)!=1 and s[-1] in punct)]
def clear_tokens(token, label):

  find_punct_start, find_punct_end = starts_or_ends_with_punctuation(token)
  if  find_punct_start==0 and find_punct_end==0: # token==punctuation mark
    return [token], [label]
  elif find_punct_start==1 and find_punct_end==0:
    return [token[0], token[1:]], ["O", label]
  elif find_punct_start==0 and find_punct_end==1:
    return [token[:-1], token[-1]], [label, "O"]
  else:
    if (len(token)==2): #случаи '#!'
       return [token[0], token[-1]], ["O", "O"]
    else:
      return [token[0], token[1:-1], token[-1]], ["O", label, "O"]
#пример
text = ['боеприпасов.', 'пресс-служба', '"В', '.', '"В.', "!#"]
labels=['SOMETHING', 'SOMETHING', 'O', 'O', 'O', 'O']
for i_text in text:
  print(f"{i_text}   ==>  {starts_or_ends_with_punctuation(i_text)}")
new_texts, new_labels = [], []
for i in range(len(text)):
  new_texts+=(clear_tokens(text[i], labels[i])[0])
  new_labels+=(clear_tokens(text[i], labels[i])[1])
print(new_texts)
print(new_labels)

for folder in folders:
  all_sequences = []
  all_labels = []
  df_folder = pd.DataFrame()
  for i_seq in dataset[f"{folder}"]:
    ann = i_seq['entities']
    txt = i_seq['text']

    '''проверка на пустые файлы'''
    if len(ann) == 0:
        continue

    '''разделение текста и токенов'''
    labels = grouped_and_sort_labeled_data(ann) #[['ORGANIZATION' 2501 2519], ....]
    split_text, ner_label = split_text_on_labeled_tokens(txt, labels)

    '''разделение текста на предложения'''
    start_iter = 0
    ner_label_sent = []
    split_text_sent =[]
    for i in range(len(ner_label)):
      if split_text[i].endswith('.') or split_text[i].endswith('!') or split_text[i].endswith('?'):
        ner_label_sent.append(ner_label[start_iter:i+1])
        split_text_sent.append(split_text[start_iter:i+1])
        start_iter = i+1

    '''очистка токенов
    почему: в разделенных словах есть знаки препинания - нужно убрать
    пример: ['"В', 'ходе', 'проведения',
    '''
    text_data, label_data =[], []
    for sent_label, sentence in zip(ner_label_sent, split_text_sent):
      new_texts, new_labels = [], []
      for i in range(len(sent_label)):
        new_texts+=(clear_tokens(sentence[i], sent_label[i])[0])
        new_labels+=(clear_tokens(sentence[i], sent_label[i])[1])
      text_data.append(new_texts)
      label_data.append(new_labels)

    # print(f"final text = {text_data}")
    # print(f"final label= {label_data}")

    df_folder = pd.concat([df_folder,pd.DataFrame({"tokens": text_data, "labels": label_data})])

  df_folder = df_folder.reset_index().drop('index', axis=1)

  # '''файлы для дообучения моделей deeppavlov'''
  # file_tr = open(f'/content/deeppavlov_tuning/{folder}.txt', "w")
  # for row in range(len(df_folder)):
  #   for i in range(len(df_folder['sequences'][row])):
  #     file_tr.writelines(f"{df_folder['sequences'][row][i]} {df_folder['labels'][row][i]}\n")
  #   file_tr.writelines(f"\n")
  # file_tr.close()

  with open(f'C:/Users/User/Desktop/PhD/changed_NEREL/{folder}_data.pickle', 'wb') as f:
    pickle.dump(df_folder, f)
    df_folder.to_csv(f"C:/Users/User/Desktop/PhD/changed_NEREL/df_{folder}.csv")
  print(f"For folder <{folder}> prepared <{df_folder.shape[0]}> sequences")

In [ ]:
file = open('C:/Users/User/Desktop/PhD/changed_NEREL/train_data.pickle', 'rb')
train_data = pickle.load(file)
file.close()

file1 = open('C:/Users/User/Desktop/PhD/changed_NEREL/test_data.pickle', 'rb')
test_data = pickle.load(file1)
file1.close()

file2 = open('C:/Users/User/Desktop/PhD/changed_NEREL/dev_data.pickle', 'rb')
dev_data = pickle.load(file2)
file2.close()

In [ ]:
entity="FAMILY"

# зануление ненужных сущностей
def replace_tags(tags, entity):
    return ['O' if tag not in ['O', f'B-{entity}', f'I-{entity}'] else tag for tag in tags]


def get_needed_entity(entity):
    few_shot_text = dev_data[dev_data['labels'].apply(lambda x: f'B-{entity}' in x)]
    few_shot_text['labels'] = few_shot_text.apply(lambda x: replace_tags(x['labels'], entity),axis=1)
    
    few_shot_text=few_shot_text.reset_index().drop("index", axis=1)
   

    few_shot_text1 = test_data[test_data['labels'].apply(lambda x: f'B-{entity}' in x)]
    few_shot_text1['labels'] = few_shot_text1.apply(lambda x: replace_tags(x['labels'], entity),axis=1)
    few_shot_text1=few_shot_text1.reset_index().drop("index", axis=1)
    

    few_shot_check = train_data[train_data['labels'].apply(lambda x: f'B-{entity}' in x)]
    few_shot_check['labels'] = few_shot_check.apply(lambda x: replace_tags(x['labels'], entity),axis=1)
    few_shot_check=few_shot_check.reset_index().drop("index", axis=1)
    return  few_shot_text, few_shot_text1, few_shot_check
few_shot_text, few_shot_text1, few_shot_check = get_needed_entity(entity)


def extract_entity(df, entity):
    entities_list=[]
    for i in range(len(df)):
        entities = []
        bool_check=False
        for i_word in range(len(df.iloc[i]['tokens'])):
            if df.iloc[i]['labels'][i_word]==f'B-{entity}':
                bool_check=True
                find_entity = df.iloc[i]['tokens'][i_word]
            elif df.iloc[i]['labels'][i_word]==f'I-{entity}':
                find_entity += " "  + df.iloc[i]['tokens'][i_word]
            elif df.iloc[i]['labels'][i_word]=='O' and bool_check==True:
                bool_check=False
                entities.append(find_entity)       
        entities_list.append(entities)         
    df["entities"] = entities_list
    df['joined_entities'] = df['entities'].apply(lambda x: ', '.join(x))
    return df

few_shot_check = extract_entity(few_shot_check, entity)
few_shot_text = extract_entity(few_shot_text, entity)
few_shot_text1 = extract_entity(few_shot_text1, entity)


def remove_repeated_subsequences(text):
    words = text.split()
    n = len(words)
    i = 0
    result = []
    
    while i < n:
        found_repetition = False
        
        for length in range(1, (n - i) // 2 + 1):
            subsequence = words[i:i + length]
            next_subsequence = words[i + length:i + 2 * length]
            if subsequence == next_subsequence:
                found_repetition = True
                i += length
                break
        
        if not found_repetition:
            result.append(words[i])
            i += 1

    return ' '.join(result)


res2 = pd.concat([few_shot_text1, few_shot_text], ignore_index=True) 
res1 = pd.concat([res2, few_shot_check], ignore_index=True) 
print(f"кол-во предложений с сущностью {entity} = {len(few_shot_check)+len(few_shot_text1)+len(few_shot_text)}")
df_example = pd.DataFrame()
labels = [5, 14, 20, 26, 37]
for i in labels:
    df_example = df_example._append(res1.iloc[i], ignore_index=True)
df_example
df_check = res1.drop(labels=labels, axis=0).reset_index().drop("index", axis=1)
df_check.to_csv("df_fewshot.csv")
df_check

In [ ]:
def remove_repeated_subsequences(text):
    words = text.split()
    n = len(words)
    i = 0
    result = []
    
    while i < n:
        found_repetition = False        
        for length in range(1, (n - i) // 2 + 1):
            subsequence = words[i:i + length]
            next_subsequence = words[i + length:i + 2 * length]
#             print(f"subsequence = {subsequence}\nnext_subsequence={next_subsequence}\n")
            if subsequence == next_subsequence:
                found_repetition = True
                i += length
                break
        
        if not found_repetition:
            result.append(words[i])
            i += 1

    return ' '.join(result)

# Example usage
# input_text = "9 апреля семья капитана Мура капитана Мура запустила от его имени кампанию по сбору средств для службы здравоохранения на платформе JustGiving"
input_text = 'были и зеленое , зеленое яблоко яблоко яблоко и апельсин и семья капитана Мура капитана Мура'
output_text = remove_repeated_subsequences(input_text)
print(output_text)

In [ ]:
system_prompt1 = """Ты находишь сущности определенной тематики в предложениях. Сущности - это слова или словосочетания, которые характеризуют определенную тематику.\
Например, сущность (словосочетание)  \"11 ноября\" характеризует тематику ДАТА. """
system_prompt2 = """Необходимо найти сущности с тематикой СЕМЬЯ. СЕМЬЯ - это сообщество, основанное на браке супругов, помимо которых включает и их детей. СЕМЬЯ состоит из слов: семья, брат, братья, сестра, сестры, клан, мама, папа, муж, жена, род, родственники, родные, близкие, родня; также ассоциируется с фамилиями (в множественном числе) - например, Ивановы, Петровы, Александр и Андрей Федоровы"""
user_prompt1 = remove_repeated_subsequences(' '.join(df_example['tokens'].iloc[0]))
assistant_prompt1 = df_example['joined_entities'].iloc[0]

user_prompt2 = remove_repeated_subsequences(' '.join(df_example['tokens'].iloc[1]))
assistant_prompt2 = df_example['joined_entities'].iloc[1]

user_prompt3 = remove_repeated_subsequences(' '.join(df_example['tokens'].iloc[2]))
assistant_prompt3 = df_example['joined_entities'].iloc[2]

user_prompt4 = remove_repeated_subsequences(' '.join(df_example['tokens'].iloc[3]))
assistant_prompt4 = df_example['joined_entities'].iloc[3]

user_prompt5 = remove_repeated_subsequences(' '.join(df_example['tokens'].iloc[4]))
assistant_prompt5 = df_example['joined_entities'].iloc[4]

user_prompt6 = remove_repeated_subsequences(' '.join(df_check['tokens'].iloc[0]))



In [ ]:
sys.stdout = orig_stdout
f.close()

In [ ]:
# orig_stdout = sys.stdout
# f = open(os.path.abspath(f"C:/Users/User/Desktop/PhD/phd/output_yandexgpt.txt"), 'w', encoding="utf-8")
# sys.stdout = f
for i in range(len(df_check)):
    user_prompt6 = remove_repeated_subsequences(' '.join(df_check['tokens'].iloc[i]))
    prompt_list=[system_prompt1, system_prompt2, user_prompt1, assistant_prompt1,
                                                 user_prompt2, assistant_prompt2,
                                                 user_prompt3, assistant_prompt3, 
                                                 user_prompt4, assistant_prompt4,
                                                 user_prompt5, assistant_prompt5,
                                                 user_prompt6]
   
    print(f"#{i+1}")
    print(f"sentance = {user_prompt6}")
    print(yandex_generate(prompt_list))
# sys.stdout = orig_stdout
# f.close()

In [ ]:
df_example